In [26]:
import pandas as pd
import sys
import numpy as np
from glob import glob
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

0. LOAD DATA

In [27]:
# read data and fill with 0s
df = pd.read_csv('Data/202001_android_data_raw.csv', index_col='Marketing_Cloud_Visitor_ID')

In [28]:
df.head(2)

,ad_pay_to_publish,ad_published,blog-detail,confirm_delete_ad,detail,detail_map,forgot_password,general,homepage,login,...,Lead_Counteroffer_eMail_madrid,Lead_View_Phone_barcelona,Lead_View_Phone_big,Lead_View_Phone_little,Lead_View_Phone_madrid,total_active_days,mean_inactive_days,max_inactive_days,min_inactive_days,n_of_user_ids
Marketing_Cloud_Visitor_ID,,,,,,,,,,,,,,,,,,,,,
00000107708722978450827756473807907358,0.0,0.0,0.0,0.0,57.0,5.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,17.0,2.0,6.0,1.0,1.0
00000136845845092372076348413986631894,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,16.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,7.0,17.0,1.0,0.0


In [55]:
print('Dataset Initial shape')
print(df.shape)
df = df.fillna(0)

Dataset Initial shape
(1092677, 123)


1.1 REMOVE ALL 0'S COLUMNS

In [56]:
# remove zero columns
features = (df != 0).any(axis=0)
print('Data set shape will be:')
print(df.loc[:, features].shape)
print('after removing zeroes columns')
# save excel file with list of features that have no data
df = df.loc[:, features]

Data set shape will be:
(1092677, 100)
after removing zeroes columns


In [31]:
# remove columns that are not actions 
df_aux = df.copy()
to_remove = ['total_active_days',
 'mean_inactive_days',
 'max_inactive_days',
 'min_inactive_days',
 'n_of_user_ids']
df_aux = df_aux.drop(columns=to_remove)

1.2 REMOVE ALL 0 USERS

In [32]:
n = len(list(df_aux.columns))
non_bounces = (df_aux == 0).sum(axis=1).to_frame(name='perc_of_zeroes')
df_aux = pd.merge(df_aux, non_bounces, how='inner', on='Marketing_Cloud_Visitor_ID')
df_aux['perc_of_zeroes'] = round ( (df_aux['perc_of_zeroes']/n)*100, 2 )
print('Dataset shape will be:')
print(df_aux[df_aux.perc_of_zeroes!=100].shape)
print('after removing rows with all 0 features')

Dataset shape will be:
(1059036, 96)
after removing rows with all 0 features


1.3 REMOVE users with very low engagement

In [34]:
df_aux = df_aux[df_aux.perc_of_zeroes<=97]
df1 = df_aux.drop(columns=['perc_of_zeroes'])
print('Dataset shape has shape:')
print(df1.shape)
print('after removing users with very low engagement')

Dataset shape has shape:
(745330, 95)
after removing users with very low engagement


2. REMOVE ad-hoc HIGH CORRELATED features

In [43]:
# correlation matrix
corr = df1.corr()
s = corr.abs().unstack()
so = s.sort_values(kind="quicksort", ascending=False)
so1 = so.to_frame(name='correlation')
so1.index = so1.index.set_names(['column1','column2'])
so1 = so1[so1.correlation!=1]
print('Pair of columns with high correlation:')
print(so1[so1.correlation>=0.80].shape[0]/2)
so1[so1.correlation>=0.80]

Pair of columns with high correlation:
9.0


,,correlation
column1,column2,
homepage,Home_Viewed,0.999996
Home_Viewed,homepage,0.999996
Form_Success,Lead_Confirmation_eMail,0.993520
Lead_Confirmation_eMail,Form_Success,0.993520
Lead_Callphone,Lead_CallphoneRENT,0.933048
Lead_CallphoneRENT,Lead_Callphone,0.933048
micrositeSALE,microsite,0.912377
microsite,micrositeSALE,0.912377
Lead_Confirmation_eMail,Lead_Confirmation_eMailRENT,0.899131


In [44]:
# We remove ad-hoc columns with high correlation
df1 = df1.drop(columns=['Home_Viewed', 'Form_Success', 'microsite', 'matches'])

3. Include frequency features and Remove cookies with multiple user ids

In [53]:
df = pd.read_csv('Data/202001_android_data_raw.csv', index_col='Marketing_Cloud_Visitor_ID')

In [68]:
df2 = pd.merge(df1, df[to_remove], on='Marketing_Cloud_Visitor_ID', how='left')

In [72]:
print('We will remove ' + 
      str(df2[df2.n_of_user_ids>=2].shape[0]) + 
      ' cookies with more than one user id')

We will remove 501 cookies with more than one user id


In [73]:
df2 = df2[df2.n_of_user_ids<=1]
print(df2.shape[0])

744829


4. SAVE THE FINAL DATAFRAME

In [74]:
print(df2.shape)

(744829, 96)


In [81]:
df2.head(2)

,ad_pay_to_publish,ad_published,blog-detail,confirm_delete_ad,detail,detail_map,forgot_password,general,homepage,login,...,Lead_Confirmation_eMail_madrid,Lead_Counteroffer_eMail_barcelona,Lead_Counteroffer_eMail_big,Lead_Counteroffer_eMail_little,Lead_Counteroffer_eMail_madrid,total_active_days,mean_inactive_days,max_inactive_days,min_inactive_days,n_of_user_ids
Marketing_Cloud_Visitor_ID,,,,,,,,,,,,,,,,,,,,,
00000107708722978450827756473807907358,0.0,0.0,0.0,0.0,57.0,5.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,17.0,2.0,6.0,1.0,1.0
00000136845845092372076348413986631894,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,16.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,7.0,17.0,1.0,0.0


In [82]:
df2.to_csv('Data/202001_android_data_cleansed.csv', index=True)

In [76]:
# save it to the EC2 folder for dimension reduction
df2.to_csv('~/Desktop/EC2/202001_android_data_cleansed.csv')